# Trabalho Final GCC108 - Teoria da Computação #

Prof.: Douglas H. S. Abreu

Nomes:
- Iago Manoel Brito de Sá Vaz da Silva
- Vinicius Caputo de Castro

Turma: 10A

Repositório: <https://github.com/iagocq/gcc108-trabalho-final>

---

# Resolução dos Exercícios

#### 1) Descreva com suas palavras uma estratégia para o desenvolvimento de uma máquina de Turing que compute a soma de 2 números binários.

No papel, a soma de números binários é realizada dígito a dígito, da direita à esquerda. No entanto, isso é um pouco inconveniente no contexto de máquinas de Turing, visto que a fita é limitada à esquerda, e na soma os resultado cresce nessa direção. Para combater isso, a máquina soma os dígitos dos argumentos na ordem natural, mas escreve o resultado logo após o segundo argumento, na ordem reversa.

Para a soma em si, o processo é o seguinte:

- Soma um par de dígitos ainda não processado e escreve o resultado. Se essa soma resulta em um carry (vai um, $1+1 = 0$ e um carry), 2 é escrito ao invés disso.

- Quando um outro par de dígitos é somado e 2 é encontrado no final do resultado, outra soma é realizada com 1. Se o resultado é maior ou igual a 10, outro carry é colocado na frente do resultado.

Para apresentar o resultado, os dois argumentos iniciais são apagados e o resultado é escrito de trás pra frente no início da fita.


#### 2) Faça o esboço por meio de desenho da máquina de Turing proposta.